# Read Arduino Sensor Data and Pass to PC through Serial 

# Reading EC-Lab .mpr file format

In [7]:
from galvani import BioLogic as BL
import pandas as pd

mpr = BL.MPRfile('test_C01.mpr')
df = pd.DataFrame(mpr.data)
print(df)

       flags      time/s  control/V     Ewe/V        I/mA   dQ/mA.h  \
0          7     0.00000   0.000000  0.028822    0.000000  0.000000   
1          7     1.00000   0.000000  0.027907    0.000000  0.000000   
2          7     2.00000   0.000000  0.027221    0.000000  0.000000   
3          7     3.00000   0.000000  0.027164    0.000000  0.000000   
4          7     4.00000   0.000000  0.026782    0.000000  0.000000   
5          7     5.00000   0.000000  0.026096    0.000000  0.000000   
6          7     6.00000   0.000000  0.025810    0.000000  0.000000   
7          7     7.00000   0.000000  0.025657    0.000000  0.000000   
8          7     8.00000   0.000000  0.024799    0.000000  0.000000   
9          7     9.00000   0.000000  0.024666    0.000000  0.000000   
10        23     9.99980  -1.175900  0.024456    0.000000  0.000000   
11        18    10.15000  -1.175899 -1.176222 -114.986916 -0.000986   
12        18    10.25000  -1.175899 -1.176241 -112.226948 -0.004103   
13    

# Biologic Python API

In [1]:
""" Bio-Logic OEM package python API.

Script shown as an example of how to interface with a Biologic instrument
in Python using the EC-Lab OEM Package library.

The script uses parameters which are provided below.

"""

import sys

import kbio.kbio_types as KBIO
from kbio.kbio_api import KBIO_api

from kbio.c_utils import c_is_64b
from kbio.utils import exception_brief

#------------------------------------------------------------------------------#

# Test parameters, to be adjusted
#address = "USB0"
address = "144.118.177.213"
channel = 1
timeout = 10

verbosity = 3
load_firmware = True

binary_path = "../../EC-Lab Development Package/EC-Lab Development Package/"
#==============================================================================#

# Helper functions

def newline () : print()
def print_exception (e) : print(f"{exception_brief(e,verbosity)}")

def print_messages (ch) :
    while True :
        msg = api.GetMessage(id_,ch)
        if not msg : break
        print(msg)

def print_USB_info (index) :
    """Print device information at USB index, if any."""
    try :
        info = api.USB_DeviceInfo(index)
        print(f"> USB{index} info : {info}")
    except Exception as e :
        print(f"> USB{index} info : {e}")
        


# determine library file according to Python version (32b/64b)

newline()

if c_is_64b :
    print("> 64b application")
    DLL_file = "EClib64.dll"
else :
    print("> 32b application")
    DLL_file = "EClib.dll"

DLL_path = binary_path + DLL_file

#==============================================================================#

"""

Example main code : excercise various OEM package API calls.

  * open the DLL,
  * display its version,
  * display connected USB devices info if any,
  * connect to the device (whose address is defined above),
  * show relevant device info,
  * test whether the connection is working,
  * enumerate the channels available for that device,
  * retrieve and show the channel info (for the channel defined above),
  * optionally load the firmware if required,
  * re-display the channel info (might have changed if firmware was not loaded before),
  * display communication speed indication,
  * print messages that the firmware has issued,
  * display and modify the instrument hardware configuration,
  * disconnect from the target,
  * show how to translate an error code to a descriptive text,
  * demonstrate that an exception is raised in case of error
   (here calling a function after being disconnected)

Note: for each call to the DLL, the base API function is shown in a comment.

"""


try :

    # API initialize
    api = KBIO_api(DLL_path)

    # BL_GetLibVersion
    version = api.GetLibVersion()
    print(f"> EcLib version: {version}")
    newline()

    # BL_GetUSBdeviceinfos
    print_USB_info(0)
    print_USB_info(1)
    newline()

    # BL_Connect
    id_, device_info = api.Connect(address,timeout)
    print(f"> device[{address}] info :")
    print(device_info)
    newline()

    # detect instrument family
    is_VMP3   = device_info.model in KBIO.VMP3_FAMILY
    is_VMP300 = device_info.model in KBIO.VMP300_FAMILY

    # BL_TestConnection
    ok = "OK" if api.TestConnection(id_) else "not OK"
    print(f"> device[{address}] connection : {ok}")
    newline()

    # BL_GetChannelsPlugged
    # .. PluggedChannels is a generator, expand into a set
    channels = {*api.PluggedChannels(id_)}
    print(f"> device[{address}] channels : {channels}")
    newline()

    # test whether the configured channel exists
    if channel not in channels :
        print(f"Configured channel {channel} does not belong to device channels {channels}")
        sys.exit(-1)

    # BL_GetChannelInfos
    channel_info = api.GetChannelInfo(id_,channel)
    print(f"> Channel {channel} info :")
    print(channel_info)
    newline()

    # based on family, determine firmware filenames
    if is_VMP3 :
        firmware_path = "kernel.bin"
        fpga_path     = "Vmp_ii_0437_a6.xlx"
    elif is_VMP300 :
        firmware_path = "kernel4.bin"
        fpga_path     = "vmp_iv_0395_aa.xlx"
    else :
        firmware_path = None

    if firmware_path :
        print(f"> Loading {firmware_path} ...")
        # create a map from channel set
        channel_map = api.channel_map({channel})
        # BL_LoadFirmware
        api.LoadFirmware(id_, channel_map, firmware=firmware_path, fpga=fpga_path, force=load_firmware)
        print("> ... firmware loaded")
        newline()

    # re-display info, as loading firmware provides more
    print(f"> Channel {channel} info :")
    # BL_GetChannelInfos
    info = api.GetChannelInfo(id_,channel)
    print(info)
    newline()

    # BL_TestCommSpeed
    rcvt_speed, firmware_speed = api.TestComSpeed(id_, channel)
    speeds = {'rcvt': rcvt_speed, 'firmware': firmware_speed}
    print(f"> device[{address}] speeds :")
    print(speeds)
    newline()

    # BL_GetMessage
    print("> Messages so far :")
    print_messages(channel)
    newline()

    # the following functions are ony valid for a VMP300
    if is_VMP300 :

        # BL_GetHardConf
        hw_conf = api.GetHardwareConf(id_, channel)
        hw_conf = { 'cnx': hw_conf.connection, 'mode': hw_conf.mode }
        print("> current hardware configuration :")
        print(hw_conf)

        # BL_SetHardConf
        cnx = KBIO.HW_CNX.WE_TO_GND
        mode = KBIO.HW_MODE.FLOATING
        api.SetHardwareConf(id_, channel, cnx.value, mode.value)

        # BL_GetHardConf
        hw_conf = api.GetHardwareConf(id_, channel)
        hw_conf = {'cnx': hw_conf.connection, 'mode': hw_conf.mode}
        print("> new hardware configuration :")
        print(hw_conf)
        newline()

        # return to a standard configuration
        cnx = KBIO.HW_CNX.STANDARD
        mode = KBIO.HW_MODE.GROUNDED
        api.SetHardwareConf(id_, channel, cnx.value, mode.value)

    # BL_Disconnect
    api.Disconnect(id_)
    print(f"> disconnected from device[{address}]")
    newline()

    # BL_GetErrorMsg
    err_code = -202
    err_msg = api.GetErrorMsg(err_code)
    print(f"> error[{err_code}] description : '{err_msg}'")
    newline()

    # id_ is no longer valid, its use will trigger an exception
    print("> an exception will be raised :")
    ok = "OK" if api.TestConnection(id_) else "not OK"

except Exception as e :
    print_exception(e)

#==============================================================================#



> 64b application
> EcLib version: 6.4.0.0

> USB0 info : no information available for USB0
> USB1 info : no information available for USB1

> device[144.118.177.213] info :
VSP 32MB, CPU=9200, 3 channels, 5 slots
Firmware: v10.44 2015/8/31
3 connections, HTdisplay on

> device[144.118.177.213] connection : OK

> device[144.118.177.213] channels : {1, 2, 5}

> Channel 1 info :
Channel: 1
C340_SP150NZ board, S/N 5267
no LC head
no EIS capabilities
0 technique
State: STOP
no amplifiers
IRange: [I_RANGE_10uA, I_RANGE_1A]
MaxBandwidth: 7
Memory: 768.0KB (0.00% filled)
KERNEL (v5.282), FPGA (A60F)

> Loading kernel.bin ...
> ... firmware loaded

> Channel 1 info :
Channel: 1
C340_SP150NZ board, S/N 5267
no LC head
no EIS capabilities
0 technique
State: STOP
no amplifiers
IRange: [I_RANGE_10uA, I_RANGE_1A]
MaxBandwidth: 7
Memory: 768.0KB (0.00% filled)
KERNEL (v5.282), FPGA (A60F)

> device[144.118.177.213] speeds :
{'rcvt': 0, 'firmware': 0}

> Messages so far :
[9] board code = 10 
[645] 

In [5]:

""" Bio-Logic OEM package python API.

Script shown as an example of how to run an experiment with a Biologic instrument
using the EC-Lab OEM Package library.

The script uses parameters which are provided below.

"""
################################################################################
################################################################################
################################################################################
import sys
import time
from dataclasses import dataclass

import kbio.kbio_types as KBIO
from kbio.kbio_api import KBIO_api

from kbio.c_utils import c_is_64b
from kbio.utils import exception_brief

from kbio.kbio_tech import ECC_parm, make_ecc_parm, make_ecc_parms, print_experiment_data
import matplotlib.pyplot as plt
import dash
import dash_core_components as dcc
import dash_html_components as html

from collections import deque
import plotly.graph_objs as go
import random

from dash.dependencies import Output, Input
import plotly
import time
import serial


################################################################################
################################################################################
################################################################################
#Required if using Jupyter Notebook %matplotlib inline
#------------------------------------------------------------------------------#

# Test parameters, to be adjusted
#address = "USB0"
address = "144.118.177.213"
channel = 1
timeout = 10

verbosity = 3
load_firmware = True

binary_path = "../../EC-Lab Development Package/EC-Lab Development Package/"
################################################################################
################################################################################
################################################################################
# ======================== Setting the margins
layout = go.Layout(
    margin=go.layout.Margin(
        l=40,  # left margin
        r=40,  # right margin
        b=10,  # bottom margin
        t=35  # top margin
    )
)
# CP parameter values

cp3_tech_file = "cp.ecc"
cp4_tech_file = "cp4.ecc"

repeat_count = 2
record_dt = 0.1  # seconds
record_dE = 0.1  # Volts
i_range = 'I_RANGE_10mA'

all_data=[] #empty list to save data

################################################################################
################################################################################
################################################################################
# dictionary of CP parameters (non exhaustive)

CP_parms = {
    'current_step':  ECC_parm("Current_step", float),
    'step_duration': ECC_parm("Duration_step", float),
    'vs_init':       ECC_parm("vs_initial", bool),
    'nb_steps':      ECC_parm("Step_number", int),
    'record_dt':     ECC_parm("Record_every_dT", float),
    'record_dE':     ECC_parm("Record_every_dE", float),
    'repeat':        ECC_parm("N_Cycles", int),
    'I_range':       ECC_parm("I_Range", int),
}
################################################################################
################################################################################
################################################################################
# defining a current step parameter

@dataclass
class current_step :
    current: float
    duration: float
    vs_init: bool = False

# list of step parameters
steps = [
  current_step(0.025, 90), # 1mA during 2s
  current_step(0.5, 90), # 2mA during 1s
  #current_step(0.0005, 3, True), # 0.5mA delta during 3s
]

#==============================================================================#
################################################################################
################################################################################
################################################################################
# helper functions

def newline () : print()
####################################################################################
def print_exception (e) : print(f"{exception_brief(e, verbosity>=2)}")
####################################################################################
def print_messages (ch) :
    """Repeatedly retrieve and print messages for a given channel."""
    while True :
        # BL_GetMessage
        msg = api.GetMessage(id_,ch)
        if not msg : break
        print(msg)
####################################################################################
def read_arduino(device):

    arduino_data = device.readline()
    
    decoded_values = str(arduino_data[0:len(arduino_data)].decode("utf-8"))
        
    return decoded_values
####################################################################################
def convert_Arduino_Data_to_Float(arduino_data):
    i = 0
    j = 0
    string = ""
    num = [None]* 19
    #extract the 24 different data points from the arduino
    while arduino_data[i] != "\n":
        string = ""
        while arduino_data[i].isspace() == False and i < (len(arduino_data) - 1):
            string = string + arduino_data[i]
            i = i + 1
        num[j] = float(string)
        j = j + 1
        while arduino_data[i].isspace() and i < (len(arduino_data) - 1):
            i = i + 1
    return num
####################################################################################
def update_graph_scatter(n):     
            data = plotly.graph_objs.Scatter(
                x=list(X),
                y=list(Y),
                name='Scatter',
                mode= 'lines+markers'
                )
            
            return {'data': [data],
            'layout' : go.Layout(title="Chronopotentiometry", xaxis=dict(title="Time (s)", range=[min(X),max(X)]),
                                 yaxis = dict(title="Voltage (V)", range = [min(Y),max(Y)]),)}
####################################################################################

device = serial.Serial('COM11', 115200)
dummy = read_arduino(device)
X = deque(maxlen=500)
X.append(0)
  
Y = deque(maxlen=500)
Y.append(0)
  
app = dash.Dash(__name__)

app.layout = html.Div(
    [
        dcc.Graph(id = 'live-graph', animate = True),
        dcc.Interval(
            id = 'graph-update',
            interval = 2000,
            n_intervals = 0
        ),
    ])

# determine library file according to Python version (32b/64b)
################################################################################
################################################################################
################################################################################
if c_is_64b :
    DLL_file = "EClib64.dll"
else :
    DLL_file = "EClib.dll"

DLL_path = binary_path + DLL_file

#==============================================================================#
################################################################################
################################################################################
################################################################################
"""

Example main :

  * open the DLL,
  * connect to the device using its address,
  * retrieve the device channel info,
  * test whether the proper firmware is running,
  * if it is, print all the messages this channel has accumulated so far,
  * create a CP parameter list (a subset of all possible parameters),
  * load the CP technique into the channel,
  * start the technique,
  * in a loop :
      * retrieve and display experiment data,
      * display messages,
      * stop when channel reports it is no longer running

Note: for each call to the DLL, the base API function is shown in a comment.

"""

try :

    newline()

    # API initialize
    api = KBIO_api(DLL_path)

    # BL_GetLibVersion
    version = api.GetLibVersion()
    print(f"> EcLib version: {version}")
    newline()

    # BL_Connect
    id_, device_info = api.Connect(address)
    print(f"> device[{address}] info :")
    print(device_info)
    newline()

    # detect instrument family
    is_VMP3 = device_info.model in KBIO.VMP3_FAMILY

    # BL_GetChannelInfos
    channel_info = api.GetChannelInfo(id_,channel)
    print(f"> Channel {channel} info :")
    print(channel_info)
    newline()

    if not channel_info.is_kernel_loaded :
        print("> kernel must be loaded in order to run the experiment")
        sys.exit(-1)

    # pick the correct ecc file based on the instrument family
    tech_file = cp3_tech_file if is_VMP3 else cp4_tech_file

    # BL_GetMessage
    print("> messages so far :")
    print_messages(channel)
    newline()
################################################################################
################################################################################
################################################################################
    # BL_Define<xxx>Parameter

    p_steps = list()

    for idx, step in enumerate(steps) :
        parm = make_ecc_parm(api, CP_parms['current_step'], step.current, idx)
        p_steps.append(parm)
        parm = make_ecc_parm(api, CP_parms['step_duration'], step.duration, idx)
        p_steps.append(parm)
        parm = make_ecc_parm(api, CP_parms['vs_init'], step.vs_init, idx)
        p_steps.append(parm)

    # number of steps is one less than len(steps)
    p_nb_steps = make_ecc_parm(api, CP_parms['nb_steps'], idx)

    # record parameters
    p_record_dt = make_ecc_parm(api, CP_parms['record_dt'], record_dt)
    p_record_dE = make_ecc_parm(api, CP_parms['record_dE'], record_dE)

    # repeating factor
    p_repeat = make_ecc_parm(api, CP_parms['repeat'], repeat_count)
    p_I_range  = make_ecc_parm(api, CP_parms['I_range'], KBIO.I_RANGE[i_range].value)

    # make the technique parameter array
    ecc_parms = make_ecc_parms(api, *p_steps, p_nb_steps, p_record_dt, p_record_dE, p_I_range, p_repeat)

    # BL_LoadTechnique
    api.LoadTechnique(id_, channel, tech_file, ecc_parms, first=True, last=True, display=(verbosity>1))

    # BL_StartChannel
    api.StartChannel(id_, channel)

################################################################################
################################################################################
################################################################################
    
    @app.callback(
            Output('live-graph', 'figure'),
            [ Input('graph-update', 'n_intervals'), ]
            )
    
    
    while True:
               
        data1 = api.GetData(id_, channel)
        status, data_save = print_experiment_data(api, data1)
            
        stringData = read_arduino(device) #get serial data from arduino
        numData = convert_Arduino_Data_to_Float(stringData) #convert arduino serial to float
               
        X.append(data_save[0])
        Y.append(data_save[1])
            
        app.layout = html.Div([
        html.H1('Website Analytics Dashboard', style={'text-align': 'center', 'background-color': '#ede9e8'}), 
            update_graph_scatter()
            ])
            
        all_data.append(data_save)
            
        time.sleep(2);
        if status == 'STOP':
                break
                print("> experiment done")
                api.Disconnect(id_)
                newline()
                # BL_Disconnect
           
         
except KeyboardInterrupt :
    print(".. interrupted")
    api.Disconnect(id_)
except Exception as e :
    print_exception(e)
    api.Disconnect(id_)

if __name__ == '__main__':
    app.run_server()

#==============================================================================#


    



SyntaxError: invalid syntax (<ipython-input-5-0e6eee2b79f0>, line 299)

In [6]:
# Standard Imports
import dash
from dash import dcc
from dash import html
import pandas as pd
import plotly.graph_objects as go

analytics = pd.DataFrame({'country': ['USA', 'UK', 'France', 'Germany', ' China', 'Pakistan', 'India'],
                          'users': [1970, 950, 760, 810, 2800, 1780, 2250],
                          'page_views': [2500, 1210, 760, 890, 3200, 1910, 2930],
                          'avg_duration': [75, 60, 63, 79, 57, 61, 72],
                          'bounce_rate': [51, 65, 77, 43, 54, 57, 51]})

# ======================== Setting the margins
layout = go.Layout(
    margin=go.layout.Margin(
        l=40,  # left margin
        r=40,  # right margin
        b=10,  # bottom margin
        t=35  # top margin
    )
)

# ======================== Plotly Graphs
def get_bar_chart():
    barChart = dcc.Graph(figure=go.Figure(layout=layout).add_trace(go.Bar(x=analytics['country'],
                                                                          y=analytics['users'],
                                                                          marker=dict(color='#351e15'))).update_layout(
        title='Users', plot_bgcolor='rgba(0,0,0,0)'),
        style={'width': '50%', 'height': '40vh', 'display': 'inline-block'})
    return barChart


def get_line_chart():
    lineChart = dcc.Graph(figure=go.Figure(layout=layout).add_trace(go.Scatter(x=analytics['country'],
                                                                               y=analytics['page_views'],
                                                                               marker=dict(
                                                                                   color='#351e15'))).update_layout(
        title='Page Views', plot_bgcolor='rgba(0,0,0,0)'),
        style={'width': '50%', 'height': '40vh', 'display': 'inline-block'})
    return lineChart


def get_scatter_plot():
    scatterPlot = dcc.Graph(figure=go.Figure(layout=layout).add_trace(go.Scatter(x=analytics['country'],
                                                                                 y=analytics['avg_duration'],
                                                                                 marker=dict(
                                                                                     color='#351e15'),
                                                                                 mode='markers')).update_layout(
        title='Average Duration', plot_bgcolor='rgba(0,0,0,0)'),
        style={'width': '50%', 'height': '40vh', 'display': 'inline-block'})
    return scatterPlot





# ======================== Dash App
app = dash.Dash(__name__)

# ======================== App Layout
app.layout = html.Div([
    html.H1('Website Analytics Dashboard', style={'text-align': 'center', 'background-color': '#ede9e8'}),
    get_bar_chart(),
    get_line_chart(),
    get_scatter_plot(),
    get_pie_chart()
])

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2022 18:37:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 18:37:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 18:37:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 18:37:34] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 18:37:34] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
